# Penguin Data
---
**Author:** Robert Kelley  
**Version:** 2.0  
**Semester:** Spring 2021  
**Summary:**  

I developed this notebook to so we could walk through the various functions from Pandas for getting descriptive statistics on a dataset.  The dataset for this notebook was obtained from: https://github.com/allisonhorst/palmerpenguins.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import scipy.stats as stats

## Read the dataset / Quick look at data

In [ ]:
df = pd.read_csv('penguins_size.csv')

In [ ]:
df.head()

In [ ]:
male = df[df['sex']=='MALE']

In [ ]:
female = df[df['sex']=='FEMALE']

In [ ]:
len(female)

In [ ]:
len(male)+len(female)

In [ ]:
df.describe()

## Measures of Centrality
How the data is clustered.

In [ ]:
df['culmen_length_mm'].median()

In [ ]:
df.head()

In [ ]:
df.mode()

### Arithmetic Mean

In [ ]:
df.mean()

In [ ]:
female['culmen_length_mm'].mean()

### Geometric Mean

In [ ]:
values = [3.5, 75, 653, 12]
mymean =sum(values)/len(values)
print(mymean)
print(stats.gmean(values))

### Harmonic Mean

In [ ]:
rates = [15000, 16000, 28000, 21000, 30000]
mymean =sum(rates)/len(rates)
print(mymean)
print(stats.hmean(rates))

## Measures of Dispersion
How spread out our data are.

### Range

In [ ]:
df['culmen_length_mm'].max()-df['culmen_length_mm'].min()

### Interquartile Range

In [ ]:
df.culmen_length_mm.quantile(.75)-df.culmen_length_mm.quantile(.25)

### Variance

In [ ]:
df.culmen_length_mm.var()

### Standard Deviation

In [ ]:
df.culmen_length_mm.std()

## Correlation

In [ ]:
df.corr()

## Using Pandas

In [ ]:
p = pd.read_csv('penguins_lter.csv')

We can look the data with .sample(), .head() or .tail()

In [ ]:
p.sample(5)

View the column names

In [ ]:
p.columns

Group by various columns in the dataset to determine how many different values are in the categorical variables.

In [ ]:
p['Species'].groupby(p['Species']).count()

Drop columns we don't need.

In [ ]:
p.drop(['Sample Number', 'Region','Stage', 'Individual ID', 'Comments'], axis=1, inplace=True)

In [ ]:
p.columns

Rename the columns to names that are easier to work with.

In [ ]:
col_names = ['study_name',
             'species',
             'island',
             'clutch_completion',
             'date_egg',
             'bill_length',
             'bill_depth',
             'flipper_length',
             'body_mass',
             'sex',
             'delta_15',
             'delta_13'
            ]

In [ ]:
p.columns = col_names

In [ ]:
p.groupby(p.species).count()

Here was my original code.  Turns out, I didn't need to specify the last three columns because they are automatically ignored.

In [ ]:
#p[['species','t1','t2','t3']] = p.species.str.split(expand=True)
#p.drop(['t1','t2','t3'],axis=1, inplace=True)

In [ ]:
p['species']=p.species.str.split(expand=True)

In [ ]:
p.species

In [ ]:
p.head()

I didn't mean to drop the measurements on the columns. Here we are renaming just specific columns in the data set.

In [ ]:
p.rename(columns={
    'bill_length':'bill_length_mm',
    'bill_depth': 'bill_depth_mm',
    'flipper_length': 'flipper_length_mm',
    'body_mass': 'body_mass_g'
}, inplace=True)

Let's look at the data types.

In [ ]:
p.dtypes

We have several strings, and float64. date_egg should be a date.

In [ ]:
p.date_egg = pd.to_datetime(p.date_egg)

In [ ]:
p.dtypes

I want to add a new column for day of the study the penguin was observed. We first subtract date_egg from the earliest date in the column, convert to string, strip off the 'days' word from the end and convert to an int.

In [ ]:
p['study_day']=p.date_egg-p.date_egg.min()
p['study_day']=p['study_day'].astype(str)
p['study_day']=p['study_day'].str[:-4]
p['study_day']=p['study_day'].astype(int)

In [ ]:
p.study_day

We can do most of manipulations with chaining (except the last one).

In [ ]:
#p['study_day']=p.date_egg-p.date_egg.min().astype(str).str[:-4]
#p['study_day']=p['study_day'].astype(int)

In [ ]:
len(p)

Let's get rid of all the rows that have missing data in certain columns.

In [ ]:
p = p[p.bill_length_mm.notna()]

In [ ]:
p = p[p.delta_15.notna()]

In [ ]:
p['sex'].groupby(p.sex).count()

Looks like we have an errant value in 'sex'. We need to remove it. **This is the code I couldn't remember to get the index of dataframe row and then drop it**

In [ ]:
i = p[p.sex=='.'].index
p.drop(i, inplace=True)

.info() gives us some summary meta data.  I noticed that we are missing the sex for 5 penguins.

In [ ]:
p.info()

In [ ]:
p.sex.groupby(p.sex.isna()).count()

Let's get the index for the rows were penguin was empty and investigate.

In [ ]:
i = p[p.sex.isna()].index
print(i)
p[p.index==9]

NaN (not a number) means its empty.  Let's just drop these too.

In [ ]:
p.drop(i, inplace=True)
p.info()

Looks good. Now we can save.

In [ ]:
p.to_csv('processed_penguins.csv', index=False)

In [ ]:
p.head()

Let's split out a separate data set for adelie penguins to save.  First, we subset the data by 'Adelie', drop the 'species' column, and then save to file.

In [ ]:
adelie = p[p['species']=='Adelie']
adelie.drop(['species'], axis=1,inplace=True)
adelie.to_csv('adelie.csv', index=False)

In [ ]:
adelie.head()

I would like to be able to save data sets for each species. This code gets the types with a groupby then it loops through the index (which contains the species) and subsets the data, drops the 'species' column and saves to a file.

In [ ]:
types = p.species.groupby(p.species).count()
for t in types.index:
    locals()[t.lower()]=p[p['species']==t]
    locals()[t.lower()].drop(['species'], axis=1,inplace=True)
    locals()[t.lower()].to_csv(t.lower()+'.csv', index=False)

## T-Test
Let's run a T-Test on the data to see if there is a significant different between mean flipper length for male and female penguins. First, we have two different ways to subset the data.

In [ ]:
male = p[p['sex']=='MALE']
female = p.query("sex == 'FEMALE'")

Then we can use ttest_ind from statsmodels.stat to do the T-Test.

In [ ]:
stats.ttest_ind(male.flipper_length_mm, female.flipper_length_mm)

The p value is less than .05 significance level so we can conclude that the mean flipper length between males and females in this sample are different.